In [2]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats

np.random.seed(123)

In [3]:
def weight_function(x: float) -> float:
    if x >= 0 and x <= 3:
        return 1 - x/3
    elif x < 0 and x >= -3:
        return 1 + x/3
    else:
        return 0

def weight_function_vectorized(x: np.ndarray) -> np.ndarray:
    return np.vectorize(weight_function)(x)

def phi_function(x: float) -> float:
    return weight_function(x)*x

def rho_scale_vectorized(x: np.ndarray) -> np.ndarray:
    return x * np.vectorize(phi_function)(x)

In [29]:
def m_estimator_loc(samples, weight_function, initial_mean_estimate = None, std_estimate=None, epsilon=0.001):
    if initial_mean_estimate is None:
        initial_mean_estimate = np.median(samples)
    if std_estimate is None:
        std_estimate = stats.median_abs_deviation(samples, scale='normal')
    
    mu_i = initial_mean_estimate
    sigma = std_estimate
    iterations_count = 0
    diff = 1

    while diff > epsilon:
        iterations_count += 1
        weights = weight_function((samples - mu_i) / sigma)
        mu_i_1 = np.sum(weights * samples) / np.sum(weights)
        diff = abs(mu_i - mu_i_1)/sigma
        mu_i = mu_i_1
    
    return mu_i, iterations_count, diff

In [30]:
def m_estimator_scale(samples, weight_function, b, mean_estimate = None, initial_std_estimate = None, epsilon=0.001):
    if mean_estimate is None:
        mean_estimate = np.median(samples)
    if initial_std_estimate is None:
        initial_std_estimate = stats.median_abs_deviation(samples, scale='normal')
    
    mu = mean_estimate
    sigma_i = initial_std_estimate
    iterations_count = 0
    diff = 1
    N = len(samples)

    while diff > epsilon:
        iterations_count += 1
        weights = weight_function((samples - mu) / sigma_i)
        sigma_i_1 = np.sqrt(1/(N*b) * np.sum(weights * (samples - mu)**2))
        diff = abs(sigma_i_1/sigma_i - 1)
        sigma_i = sigma_i_1
        
    return sigma_i, iterations_count, diff

In [ ]:
_, axes = plt.subplots(nrows=1, ncols=2, figsize=(10, 3))

x_array = np.linspace(-4, 4, 1000)
axes[0].plot(x_array, [weight_function(x) for x in x_array], label='$w(x)$')
axes[1].plot(x_array, [phi_function(x) for x in x_array], label='$\phi(x)$', color='red')

axes[0].set_title('Weight function')
axes[0].set_xlabel('x')
axes[0].set_ylabel('w(x)')
axes[0].legend()
axes[0].grid()

axes[1].set_title('Basis function')
axes[1].set_xlabel('x')
axes[1].set_ylabel('$\phi(x)$')
axes[1].legend()
axes[1].grid()

plt.show()

In [ ]:
samples_normal_dist = np.random.normal(0, 1, 1000)

# monte carlo integration for the expected value of the scale function
b = rho_scale_vectorized(samples_normal_dist).mean()

print(f'Expected value of the scale function: b = {b}')

In [8]:
epsilon = 0.001

y_array = np.array([1.03, 0.72, -0.30, 0.29, -5.78])

In [ ]:
mu_0 = np.median(y_array)
std_0 = stats.median_abs_deviation(y_array, scale='normal')

print(f'Median: {mu_0}')
print(f'Median absolute deviation: {std_0}')

In [ ]:
print(f'Estimated location parameter: {mu_i}')
print(f'Number of iterations: {iterations_count}')
print(f'Error: {diff}')

In [ ]:
scale, iterations_count, diff = m_estimator_scale(y_array, weight_function_vectorized, b)

print(f'Estimated scale: {scale}')
print(f'Number of iterations: {iterations_count}')
print(f'Error: {diff}')

In [ ]:
loc, iterations_count, diff = m_estimator_loc(y_array, weight_function_vectorized, std_estimate=std)

print(f'Estimated location parameter: {loc}')
print(f'Number of iterations: {iterations_count}')
print(f'Error: {diff}')

In [ ]:
loc, iterations_count, diff = m_estimator_loc(y_array, weight_function_vectorized)

print(f'Estimated location parameter: {loc}')
print(f'Number of iterations: {iterations_count}')
print(f'Error: {diff}')